# LiteLLM + AWS Bedrock MCP Integration Lab

This lab demonstrates how to use **LiteLLM** as a unified interface to access **AWS Bedrock** models (specifically Claude 3.5 Sonnet) with **MCP (Model Context Protocol)** servers.

## Key Benefits:
- 🔄 **Unified API**: LiteLLM provides OpenAI-compatible interface for 100+ models
- 💰 **Cost Effective**: AWS Bedrock often 60-70% cheaper than OpenAI
- 🔒 **Privacy**: Data stays in your AWS account
- 🌍 **Regional**: Using EU region models

## Setup Required:
1. AWS credentials with Bedrock access
2. Claude 3.5 Sonnet model access in AWS Bedrock
3. Node.js for MCP servers

## Import Required Libraries

In [ ]:
# Import required libraries and AWS utilities
import asyncio
import os
from agents import Agent, Runner, trace, set_tracing_disabled
from agents.mcp import MCPServerStdio

# Import AWS utilities from common module
import sys
sys.path.append('..')
from aws_utils import setup_aws_credentials, test_bedrock_connectivity

# Disable OpenAI tracing since we're using Bedrock
set_tracing_disabled(True)
print("🚫 OpenAI tracing disabled (using Bedrock instead)")

In [ ]:
# Set up AWS credentials and test Bedrock connectivity
aws_access_key, aws_secret_key, aws_session_token, aws_region = setup_aws_credentials('eu-west-1')

# Model configuration
model_id = "eu.anthropic.claude-3-5-sonnet-20240620-v1:0"
print(f"\n🤖 Using model: {model_id}")

# Test Bedrock connectivity if credentials were retrieved
if aws_access_key:
    test_bedrock_connectivity(aws_access_key, aws_secret_key, aws_session_token, aws_region, model_id)
else:
    print("⚠️ Skipping Bedrock connectivity test due to missing credentials")

In [11]:
# Set up MCP server configurations
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}
playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

print(f"📁 Sandbox path: {sandbox_path}")
print("🔧 MCP server configurations ready")

📁 Sandbox path: /Users/yongqiwu/code/agents/litellm/sandbox
🔧 MCP server configurations ready


In [12]:
# Test Fetch MCP Server
async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

print("🌐 Fetch tools available:")
fetch_tools

🌐 Fetch tools available:


[Tool(name='fetch', description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get the actual H

In [13]:
# Test Playwright MCP Server
async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

print("🎭 Playwright tools available:")
playwright_tools

🎭 Playwright tools available:


[Tool(name='browser_close', description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_resize', description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_console_messages', description='Returns all console messages', inputSchema={'type': 'object', 'properties': {}, 'additi

In [14]:
# Test Filesystem MCP Server
async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

print("📁 File tools available:")
file_tools

📁 File tools available:


[Tool(name='read_file', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=None),
 Tool(name='read_text_file', description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only work

In [15]:
# Agent instructions
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""

print("📋 Agent instructions configured")
print(f"🎯 Model: litellm/bedrock/{model_id}")

📋 Agent instructions configured
🎯 Model: litellm/bedrock/eu.anthropic.claude-3-5-sonnet-20240620-v1:0


In [16]:
# Agent instructions
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""

print("📋 Agent instructions configured")
print(f"🎯 Model: litellm/bedrock/{model_id}")

📋 Agent instructions configured
🎯 Model: litellm/bedrock/eu.anthropic.claude-3-5-sonnet-20240620-v1:0


## Run the Agent Task

Let's run the same Banoffee Pie task as the original lab, but using **Claude 3.5 Sonnet via AWS Bedrock**:

In [19]:
async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        
        print("✅ MCP servers started successfully")
        
        # Create agent with LiteLLM + Bedrock
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model=f"litellm/bedrock/{model_id}",  # Using Claude 3.5 Sonnet via Bedrock
            mcp_servers=[mcp_server_files, mcp_server_browser]
        )
        
        print("🤖 Agent created with Claude 3.5 Sonnet via AWS Bedrock")
        print("📝 Task: Find a great recipe for Banoffee Pie, then summarize it in markdown")
        
        # Run the task (tracing disabled in imports)
        # Note: OpenAI Agents SDK tracing is designed for OpenAI's platform, but we're using AWS Bedrock
        # So we disabled tracing to avoid "OPENAI_API_KEY not set" warnings since we don't need OpenAI for this demo
        result = await Runner.run(
            agent, 
            "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md"
        )
            
        print("\n" + "="*50)
        print("✅ TASK COMPLETED!")
        print("="*50)
        print(result.final_output)

✅ MCP servers started successfully
🤖 Agent created with Claude 3.5 Sonnet via AWS Bedrock
📝 Task: Find a great recipe for Banoffee Pie, then summarize it in markdown

✅ TASK COMPLETED!
Great! The Banoffee Pie recipe has been successfully summarized in markdown format and saved to the file "banoffee.md" in the "sandbox/recipes" directory. The content of the file looks correct and well-formatted.

To summarize what we've accomplished:

1. We searched for a Banoffee Pie recipe on BBC Good Food.
2. We found a great recipe and summarized it in markdown format.
3. We created a "recipes" directory within the "sandbox" folder.
4. We saved the markdown summary as "banoffee.md" in the "sandbox/recipes" directory.
5. We verified the contents of the file to ensure it was saved correctly.

The task is now complete. You have a nicely formatted Banoffee Pie recipe saved in markdown format, which you can easily read, share, or use for future reference.


## Check the Results

Let's see what file was created:

In [18]:
# Check if banoffee.md was created
import os

banoffee_file = os.path.join(sandbox_path, "banoffee.md")
if os.path.exists(banoffee_file):
    print("📄 banoffee.md created successfully!")
    with open(banoffee_file, 'r') as f:
        content = f.read()
    print("\n📖 Content:")
    print(content[:500] + "..." if len(content) > 500 else content)
else:
    print("❌ banoffee.md not found")
    print(f"📁 Files in sandbox: {os.listdir(sandbox_path) if os.path.exists(sandbox_path) else 'Sandbox not found'}")

📄 banoffee.md created successfully!

📖 Content:
# Banoffee Pie Recipe

## Description
This easy and delicious Banoffee Pie combines a crunchy graham cracker crust, soft and sweet dulce de leche, thick slices of fresh banana, and a mountain of billowy whipped cream.

## Ingredients

### For the Crust:
- 1 1/2 cups (180g) graham cracker crumbs (about 12 full sheet graham crackers)
- 1/4 cup (50g) granulated sugar
- 6 Tablespoons (85g) unsalted butter, melted

### For the Filling:
- 1 1/4 cups dulce de leche
- 2 large bananas, sliced

### For th...


## Summary

🎉 **Congratulations!** You've successfully:

1. ✅ Set up **LiteLLM** as a unified interface
2. ✅ Connected to **AWS Bedrock** Claude 3.5 Sonnet
3. ✅ Used **MCP servers** for web browsing and file operations
4. ✅ Created an agent that can browse the web and save results

## Key Differences from OpenAI:
- 🔄 **Model**: `litellm/bedrock/{model_id}` instead of `gpt-4.1-mini`
- 🔑 **Auth**: AWS credentials instead of OpenAI API key
- 💰 **Cost**: Potentially 60-70% cheaper
- 🌍 **Region**: Using EU region for data compliance

## Next Steps:
- Try different Bedrock models (Nova, Llama, etc.)
- Experiment with other MCP servers
- Set up cost monitoring in AWS
- Check tracing at: https://platform.openai.com/traces